In [1]:
#This version (v.2.3) includes initialization of pressure + updated report format
#This version (v.2.4) includes timestep design

%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
import xlrd

#Gas properties generator
p_sc = 14.7
T_sc = 520
Mw_air = 28.97
R_const = 10.732
Mw_gas = 22.836
SGg = Mw_gas/Mw_air
T_res = 190 + 460

def zg(P,T,SGg):
    
    P_pr = P/P_critical(SGg)
    T_pr = T/T_critical(SGg)
    
    #P_pr = P/738.44
    #T_pr = T/418.38
    
    A1 = 0.3265         
    A2 = -1.07          
    A3 = -0.5339        
    A4 = 0.01569        
    A5 = -0.05165        
    A6 = 0.5475
    A7 = -0.7361       
    A8 = 0.1844         
    A9 = 0.1056         
    A10 = 0.6134        
    A11 = 0.721
        
    C1 = A1 + A2 / T_pr + A3 / (T_pr * T_pr * T_pr) + A4 / (T_pr * T_pr * T_pr * T_pr) + A5 / (T_pr * T_pr * T_pr * T_pr * T_pr)
    C2 = A6 + A7 / T_pr + A8 / (T_pr * T_pr)
    C3 = A9 * (A7 / T_pr + A8 / (T_pr * T_pr))

    Ze = 1
    for i in range(0,100):
        rho_pr = 0.27 * (P_pr / (Ze * T_pr))
        C4 = A11 * rho_pr * rho_pr
        Zc = 1 + C1 * rho_pr + C2 * rho_pr * rho_pr - C3 * (rho_pr * rho_pr * rho_pr * rho_pr * rho_pr) + A10 * (1 + C4) * (rho_pr * rho_pr / (T_pr * T_pr * T_pr)) * np.exp(-C4)
        if (np.abs(Ze - Zc) < 0.001):
            break
        else:
            fz = Ze - Zc
            dfdZ = 1 + (rho_pr * rho_pr / Ze) * (C1 / rho_pr + 2 * C2 - 5 * C3 * (rho_pr * rho_pr * rho_pr) + 2 * A10 * np.exp(-C4) * (1 + C4 - C4 * C4) / (T_pr * T_pr * T_pr))
            Ze = Ze - fz / dfdZ
    
    return Zc


def T_critical(SGg):  #Sutton
    return 169.2 + 349.5 * SGg - 74 * SGg * SGg 
def P_critical(SGg): #Sutton
    return 756.8 - 131 * SGg - 3.6 * SGg**2
def T_reduced(T,SGg): #T in Rankine
    return T/T_critical(SGg)
def P_reduced(P,SGg): #T in Rankine
    return P/P_critical(SGg)

def cal_rhog(P,T,SGg):
    Mwg = SGg * Mw_air
    z = zg(P,T,SGg)
    return P*Mwg/z/R_const/T

def cal_viscg(P,T,SGg):
    z = zg(P,T,SGg)
    gasdensPT = cal_rhog(P,T,SGg)
    gasdensPT = gasdensPT * 1000 / (2.20462262184878 * ((0.3048 * 100) ** 3)) 
    Mwg = SGg * Mw_air
    XK = (9.379 + 0.01607 * Mwg) * ((T ** 1.5) / (209.2 + 19.26 * Mwg + T))
    X = 3.448 + 986.4 / T + 0.01009 * Mwg
    Y = 2.447 - 0.2224 * X
    return (XK / 10000) * np.exp(X * (gasdensPT ** Y))

def cal_cg(P,T,SGg):
    dP = 1
    z_left2 = zg(P-2*dP,T,SGg)
    z_left1 = zg(P-dP,T,SGg)
    z_cen = zg(P,T,SGg)
    z_right1 = zg(P+dP,T,SGg)
    z_right2 = zg(P+2*dP,T,SGg)
    dzdP = (z_right1-z_left1)/2/dP
    #dzdP = (-z_left2+8*z_left1-8*z_right1+z_right2)/12/dP
    return 1/P-1/z_cen*dzdP
    
def cal_Bg(P,T,SGg):
    Bg = p_sc/T_sc*zg(P,T,SGg)*T/P
    return Bg/5.6146

print(zg(5000,T_res,SGg))
print(cal_Bg(5000,T_res,SGg))
print(cal_viscg(5000,T_res,SGg))
print(cal_rhog(5000,T_res,SGg))
print(cal_rhog(14.7,520,SGg))
print(cal_cg(5000,T_res,SGg))

0.987144768022054
0.0006461292028783971
0.031758665397174306
16.581169923885977
0.06036222967048886
0.00011341287742464846


In [2]:
AA = np.zeros((3,3,3))
AA[1,2,0] = 4
print(AA)

[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [4. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]


In [3]:
def pressure_initializer(Z,p_datum,Z_datum):
    p_mid = p_datum
    dp_dZ_mid =  cal_rhog(p_mid,T_res,SGg)/144
    p_cal = p_datum + dp_dZ_mid*(Z-Z_datum)
    p_mid = (p_datum+p_cal)/2

    while True:
        dp_dZ_mid =  cal_rhog(p_mid,T_res,SGg)/144
        p_cal_new = p_datum + dp_dZ_mid*(Z-Z_datum)
        if abs(p_cal_new-p_cal) < 0.0001:
            break
        p_cal = p_cal_new
        p_mid = (p_datum+p_cal_new)/2
    return p_cal_new

print(pressure_initializer(9270,7750,9300))

7745.7738022279755


In [4]:

def cal_phi(p,p_ref,phi_ref,cr):
    return phi_ref*(1+cr*(p-p_ref))

def id_tran(iy,ix):
    return ix + Nx*iy

def Jw_cal(dx,dy,kx,ky,h,viscg,Bg,rw,skin):
    req = 0.28*np.sqrt(np.sqrt(ky/kx)*dx**2+np.sqrt(kx/ky)*dy**2)/((ky/kx)**0.25+(kx/ky)**0.25)
    return 2*np.pi*cbeta*np.sqrt(kx*ky)*h/viscg/Bg/(np.log(req/rw)+skin)

def p_res_average_cal(p):
    # MBE check
    p_res_num = 0
    p_res_deno = 0
    for ix in range (0,Nx):
        for iy in range(0,Ny):
            if (null[iz,iy,ix] != 0):
                p_res_num += dx[iz,iy,ix]*dy[iz,iy,ix]*h[iz,iy,ix]*cal_phi(p[iz,iy,ix],p_ref,phi_ref[iz,iy,ix],cr)*p[iz,iy,ix]
                p_res_deno += dx[iz,iy,ix]*dy[iz,iy,ix]*h[iz,iy,ix]*cal_phi(p[iz,iy,ix],p_ref,phi_ref[iz,iy,ix],cr)
    return p_res_num/p_res_deno

Nx = 12
Ny = 9
Nz = 3
t = 5
dt = 0.5 #days

# reservoir and PVT properties
pi = 7750
p_ref = 14.7
cr = 0 #5e-8
calpha = 5.6146
cbeta = 1.127e-3
Rconst = 10.732
Z_datum = 9300

dx = np.zeros((Nz,Ny,Nx))
dy = np.zeros((Nz,Ny,Nx))
kx = np.zeros((Nz,Ny,Nx))
ky = np.zeros((Nz,Ny,Nx))
Ax = np.zeros((Nz,Ny,Nx))
Ay = np.zeros((Nz,Ny,Nx))
phi = np.zeros((Nz,Ny,Nx))
phi_ref = np.zeros((Nz,Ny,Nx))
h = np.zeros((Nz,Ny,Nx))
top = np.zeros((Nz,Ny,Nx))
bottom = np.zeros((Nz,Ny,Nx))
middle = np.zeros((Nz,Ny,Nx))
null = np.zeros((Nz,Ny,Nx))
null_im = []

p = np.zeros((Nz,Ny,Nx))
p_new = np.zeros((Nz,Ny,Nx))
p_inner = np.zeros((Nz,Ny,Nx))
qgsc = np.zeros((Nz,Ny,Nx))
#pwf = []
pwf = np.zeros((Nz,Ny,Nx))
Jw = np.zeros((Nz,Ny,Nx))

#initialization
res_Excel = xlrd.open_workbook('CH8_reservoir prop.xlsx')

for ix in range (0,Nx):
    for iy in range(0,Ny):       
        for iz in range(0,Nz):
            dx[:,iy,ix] = res_Excel.sheet_by_name('dx').cell(iy,ix).value
            dy[:,iy,ix] = res_Excel.sheet_by_name('dy').cell(iy,ix).value

            if iz == 0:

                kx[iz,iy,ix] = res_Excel.sheet_by_name('kx1').cell(iy,ix).value
                ky[iz,iy,ix] = res_Excel.sheet_by_name('ky1').cell(iy,ix).value
                phi[iz,iy,ix] = res_Excel.sheet_by_name('phi1').cell(iy,ix).value
                phi_ref[iz,iy,ix] = res_Excel.sheet_by_name('phi1').cell(iy,ix).value
                top[iz,iy,ix] = res_Excel.sheet_by_name('T1').cell(iy,ix).value
                bottom[iz,iy,ix] = res_Excel.sheet_by_name('T2').cell(iy,ix).value
                null[iz,iy,ix] = res_Excel.sheet_by_name('null1').cell(iy,ix).value

            if iz == 1:

                kx[iz,iy,ix] = res_Excel.sheet_by_name('kx2').cell(iy,ix).value
                ky[iz,iy,ix] = res_Excel.sheet_by_name('ky2').cell(iy,ix).value
                phi[iz,iy,ix] = res_Excel.sheet_by_name('phi2').cell(iy,ix).value
                phi_ref[iz,iy,ix] = res_Excel.sheet_by_name('phi2').cell(iy,ix).value
                top[iz,iy,ix] = res_Excel.sheet_by_name('T2').cell(iy,ix).value
                bottom[iz,iy,ix] = res_Excel.sheet_by_name('T3').cell(iy,ix).value
                null[iz,iy,ix] = res_Excel.sheet_by_name('null2').cell(iy,ix).value

            if iz == 2:

                kx[iz,iy,ix] = res_Excel.sheet_by_name('kx3').cell(iy,ix).value
                ky[iz,iy,ix] = res_Excel.sheet_by_name('ky3').cell(iy,ix).value
                phi[iz,iy,ix] = res_Excel.sheet_by_name('phi3').cell(iy,ix).value
                phi_ref[iz,iy,ix] = res_Excel.sheet_by_name('phi3').cell(iy,ix).value
                top[iz,iy,ix] = res_Excel.sheet_by_name('T3').cell(iy,ix).value
                bottom[iz,iy,ix] = res_Excel.sheet_by_name('B3').cell(iy,ix).value
                null[iz,iy,ix] = res_Excel.sheet_by_name('null3').cell(iy,ix).value


            h[iz,iy,ix] = bottom[iz,iy,ix]-top[iz,iy,ix]
            middle[iz,iy,ix] = (top[iz,iy,ix] + bottom[iz,iy,ix])/2
            Ax[iz,iy,ix] = dy[iz,iy,ix]*h[iz,iy,ix]
            Ay[iz,iy,ix] = dx[iz,iy,ix]*h[iz,iy,ix]

            if null[iz,iy,ix] == 1:
                #pressure distribution (initialization)
                p[iz,iy,ix] = pressure_initializer(middle[iz,iy,ix],pi,Z_datum)
            
print(p)

[[[   0.         7747.88679396 7747.88679396 7747.88679396    0.
      0.            0.            0.            0.            0.
      0.            0.        ]
  [7747.88679396 7747.88679396 7747.88679396 7747.88679396 7747.88679396
      0.            0.            0.         7747.88679396 7747.88679396
      0.            0.        ]
  [7747.88679396 7747.88679396 7747.88679396 7747.88679396 7747.88679396
   7747.88679396 7747.88679396 7747.88679396 7747.88679396 7747.88679396
   7747.88679396 7747.88679396]
  [7747.88679396 7747.88679396 7747.88679396 7747.88679396 7747.88679396
   7747.88679396 7747.88679396 7747.88679396 7747.88679396 7747.88679396
   7747.88679396 7747.88679396]
  [7747.88679396 7747.88679396 7747.88679396 7747.88679396 7747.88679396
   7747.88679396 7747.88679396 7747.88679396 7747.88679396 7747.88679396
      0.            0.        ]
  [   0.            0.            0.         7747.88679396 7747.88679396
   7747.88679396 7747.88679396 7747.88679396 7747.886

In [5]:
# Well info
#W2
qgsc[0,2,8] = -3e+6
#W3
pwf[0,3,3] = 6000
rw_3_3 = 0.25
rw_2_8 = 0.25


#reporting-general settings
#N_step = 68
#dt_report = [0] + [0.0625]*4 + [0.125]*2 + [0.25]*4 + [0.5]*17 + [1]*40

#N_step = 18
#dt_report = [0,0.0625,0.0625,0.0625,0.0625,0.125,0.125,0.25,0.25,0.25,0.25,0.5,0.5,0.5,0.5,0.5,0.5,0.5]

N_step = 16
dt_report = [0,0.125,0.125,0.125,0.125,0.25,0.25,0.25,0.25,0.5,0.5,0.5,0.5,0.5,0.5,0.5]

#N_step = 14
#dt_report = [0,0.25,0.25,0.25,0.25,0.25,0.25,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5]

#N_step = 11
#dt_report = [0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5]
t_report = np.zeros(N_step)
p_res_average = np.zeros(N_step)
MB_error = np.zeros(N_step)


#reporting-production settings
p_28 = np.zeros(N_step)
Jw_28 = np.zeros(N_step)
pwf_28 = np.zeros(N_step)

p_33 = np.zeros(N_step)
Jw_33 = np.zeros(N_step)
q_33 = np.zeros(N_step)


p_28[0] = p[0,2,8]

#plot settings
plt_max = 4000
plt_min = 3970

        
t_cum = 0
viscg = np.zeros((Nz,Ny,Nx))
Bg = np.zeros((Nz,Ny,Nx))
rhog = np.zeros((Nz,Ny,Nx))
cg = np.zeros((Nz,Ny,Nx))
# tran_W = np.zeros((Nz,Ny,Nx))
# tran_E = np.zeros((Nz,Ny,Nx))
# tran_S = np.zeros((Nz,Ny,Nx))
# tran_N = np.zeros((Nz,Ny,Nx))
# tran_C = np.zeros((Nz,Ny,Nx))
# tau_term = np.zeros((Nz,Ny,Nx))
# Q_term = np.zeros((Nz,Ny,Nx))

# tran_Wg = np.zeros((Nz,Ny,Nx))
# tran_Eg = np.zeros((Nz,Ny,Nx))
# tran_Sg = np.zeros((Nz,Ny,Nx))
# tran_Ng = np.zeros((Nz,Ny,Nx))
# tran_Cg = np.zeros((Nz,Ny,Nx))
# Qg_term = np.zeros((Nz,Ny,Nx))

# tran_W_const = np.zeros((Nz,Ny,Nx))
# tran_E_const = np.zeros((Nz,Ny,Nx))
# tran_S_const = np.zeros((Nz,Ny,Nx))
# tran_N_const = np.zeros((Nz,Ny,Nx))

A_matrix = np.zeros((Nx*Ny,Nx*Ny))
B_matrix = np.zeros((Nx*Ny,1))
soln_matrix = np.zeros((Nx*Ny,1))

A_matrix_R = np.zeros((Nx*Ny,Nx*Ny))
B_matrix_R = np.zeros((Nx*Ny,1))
soln_matrix_R = np.zeros((Nx*Ny,1))
residual_dp = np.zeros((Nz,Ny,Nx))


#########################
# loop calculation


#Residual function (R)

def R(iy,ix,diff_dir):
    
    #temporary
    iz = 0
    
    
    
    R_e = 0.00001
    if diff_dir == 'C':
        p_C = p_inner[iz,iy,ix] + R_e
    else:
        p_C = p_inner[iz,iy,ix] 
        
    viscg_C = cal_viscg(p_C,T_res,SGg)
    Bg_C = cal_Bg(p_C,T_res,SGg)
    rhog_C = cal_rhog(p_C,T_res,SGg)
    cg_C = cal_cg(p_C,T_res,SGg)
    
    if (ix == 0) or (null[iz,iy,ix]==0):
        p_W = 0
        viscg_W = 0
        Bg_W = 0
        rhog_W = 0
        tran_W_R = 0
        tran_Wg_R = 0
    else:
        if diff_dir == 'W':
            p_W = p_inner[iz,iy,ix-1] + R_e
        else:
            p_W = p_inner[iz,iy,ix-1]
            
        viscg_W = cal_viscg(p_W,T_res,SGg)
        Bg_W = cal_Bg(p_W,T_res,SGg)
        rhog_W = cal_rhog(p_W,T_res,SGg)
        tran_W_R_const = cbeta*2/(dx[iz,iy,ix-1]/Ax[iz,iy,ix-1]/kx[iz,iy,ix-1]+dx[iz,iy,ix]/Ax[iz,iy,ix]/kx[iz,iy,ix])
        tran_W_R = tran_W_R_const/(viscg_W+viscg_C)*2/(Bg_W+Bg_C)*2
        tran_Wg_R = tran_W_R_const*(rhog_W+rhog_C)/2/144
        
    if (ix == Nx-1) or (null[iz,iy,ix]==0):
        p_E = 0
        viscg_E = 0
        Bg_E = 0
        rhog_E = 0
        tran_E_R = 0
        tran_Eg_R = 0
    else:
        if diff_dir == 'E':
            p_E = p_inner[iz,iy,ix+1] + R_e
        else:
            p_E = p_inner[iz,iy,ix+1] 

        viscg_E = cal_viscg(p_E,T_res,SGg)
        Bg_E = cal_Bg(p_E,T_res,SGg)
        rhog_E = cal_rhog(p_E,T_res,SGg)
        tran_E_R_const = cbeta*2/(dx[iz,iy,ix+1]/Ax[iz,iy,ix+1]/kx[iz,iy,ix+1]+dx[iz,iy,ix]/Ax[iz,iy,ix]/kx[iz,iy,ix])
        tran_E_R = tran_E_R_const/(viscg_E+viscg_C)*2/(Bg_E+Bg_C)*2
        tran_Eg_R = tran_E_R_const*(rhog_E+rhog_C)/2/144
    
    if (iy == 0) or (null[iz,iy,ix]==0):
        p_N = 0
        viscg_N = 0
        Bg_N = 0
        rhog_N = 0
        tran_N_R = 0
        tran_Ng_R = 0
    else:
        if diff_dir == 'N':
            p_N = p_inner[iz,iy-1,ix] + R_e
        else:
            p_N = p_inner[iz,iy-1,ix]
            
        viscg_N = cal_viscg(p_N,T_res,SGg)
        Bg_N = cal_Bg(p_N,T_res,SGg)
        rhog_N = cal_rhog(p_N,T_res,SGg)
        tran_N_R_const = cbeta*2/(dx[iz,iy-1,ix]/Ax[iz,iy-1,ix]/kx[iz,iy-1,ix]+dx[iz,iy,ix]/Ax[iz,iy,ix]/kx[iz,iy,ix])
        tran_N_R = tran_N_R_const/(viscg_N+viscg_C)*2/(Bg_N+Bg_C)*2
        tran_Ng_R = tran_N_R_const*(rhog_N+rhog_C)/2/144
    
    if (iy == Ny-1) or (null[iz,iy,ix]==0):
        p_S = 0
        viscg_S = 0
        Bg_S = 0
        rhog_S = 0
        tran_S_R = 0
        tran_Sg_R = 0
    else:
        if diff_dir == 'S':
            p_S = p_inner[iz,iy+1,ix] + R_e
        else:
            p_S = p_inner[iz,iy+1,ix]
            
        viscg_S = cal_viscg(p_S,T_res,SGg)
        Bg_S = cal_Bg(p_S,T_res,SGg)
        rhog_S = cal_rhog(p_S,T_res,SGg)
        tran_S_R_const = cbeta*2/(dx[iz,iy+1,ix]/Ax[iz,iy+1,ix]/kx[iz,iy+1,ix]+dx[iz,iy,ix]/Ax[iz,iy,ix]/kx[iz,iy,ix])
        tran_S_R = tran_S_R_const/(viscg_S+viscg_C)*2/(Bg_S+Bg_C)*2
        tran_Sg_R = tran_S_R_const*(rhog_S+rhog_C)/2/144

    
    Jw_C = 0
    #only for grid(3,3)
    if iy == 3 and ix ==3:
        Jw_C = Jw_cal(dx[0,3,3],dy[0,3,3],kx[0,3,3],ky[0,3,3],h[0,3,3],viscg_C,Bg_C,rw_3_3,0)
    
    
    tau_term_R = dx[iz,iy,ix]*dy[iz,iy,ix]*h[iz,iy,ix]*phi[iz,iy,ix]*cg_C/calpha/Bg_C
    tran_C_R = -(tran_W_R + tran_E_R + tran_S_R + tran_N_R + tau_term_R /dt + Jw_C)
    tran_Cg_R = -(tran_Wg_R + tran_Eg_R + tran_Sg_R + tran_Ng_R)
    
    Qg_term_R = 0
    if ix != 0:
        Qg_term_R += middle[iz,iy,ix-1]*tran_Wg_R
    if ix != Nx-1:
        Qg_term_R += middle[iz,iy,ix+1]*tran_Eg_R
    if iy != 0:
        Qg_term_R += middle[iz,iy-1,ix]*tran_Ng_R
    if iy != Ny-1:  
        Qg_term_R += middle[iz,iy+1,ix]*tran_Sg_R
                
    Qg_term_R += middle[iz,iy,ix]*tran_Cg_R
    Q_term_R = -(tau_term_R/dt*p[iz,iy,ix] + qgsc[iz,iy,ix] + pwf[iz,iy,ix] * Jw_C - Qg_term_R)

    sum_R = Q_term_R - (p_W*tran_W_R + p_E*tran_E_R + p_N*tran_N_R + p_S*tran_S_R + p_C*tran_C_R)
    
    if (null[iz,iy,ix]==0):
        sum_R = 0
    return sum_R

#Residual-derivative (dR/dP)
def dR_dp(iy,ix,diff_dir):
    
    R_e = 0.00001
    
    if diff_dir == 'C':
        dR_dp = (R(iy,ix,'C')-R(iy,ix,'0'))/R_e
    if diff_dir == 'W':
         dR_dp = (R(iy,ix,'W')-R(iy,ix,'0'))/R_e
    if diff_dir == 'E':
         dR_dp = (R(iy,ix,'E')-R(iy,ix,'0'))/R_e
    if diff_dir == 'N':
         dR_dp = (R(iy,ix,'N')-R(iy,ix,'0'))/R_e
    if diff_dir == 'S':
         dR_dp = (R(iy,ix,'S')-R(iy,ix,'0'))/R_e  
    return dR_dp

 ##########################  
#SIMULATOR-MATRIX SOLVER

#timestep zero

report_counter = 0

viscg[iz,iy,ix] = cal_viscg(p[iz,iy,ix],T_res,SGg)
rhog[iz,iy,ix] = cal_rhog(p[iz,iy,ix],T_res,SGg)
cg[iz,iy,ix] = cal_cg(p[iz,iy,ix],T_res,SGg)
Bg[iz,iy,ix] = cal_Bg(p[iz,iy,ix],T_res,SGg)
phi[iz,iy,ix] = cal_phi(p[iz,iy,ix],p_ref,phi_ref[iz,iy,ix],cr)
Jw[0,3,3] = Jw_cal(dx[0,3,3],dy[0,3,3],kx[0,3,3],ky[0,3,3],h[0,3,3],viscg[0,3,3],Bg[0,3,3],rw_3_3,0)

p_res_average[report_counter] = p_res_average_cal(p)
t_report[report_counter] = 0

p_28[report_counter] = p[0,2,8]
Jw_28[report_counter] = Jw_cal(dx[0,2,8],dy[0,2,8],kx[0,2,8],ky[0,2,8],h[0,2,8],viscg[0,2,8],Bg[0,2,8],rw_2_8,0)
pwf_28[report_counter] = p[0,2,8]
    
p_33[report_counter] = p[0,3,3]
q_33[report_counter] = 0
Jw_33[report_counter] = Jw[0,3,3]

t_cum = 0
report_counter = 1

while report_counter <= N_step-1:

    #temporary
    iz = 0
    
    
    dt = dt_report[report_counter]  #assign time step as specified in the array
    t_cum += dt
    t_report[report_counter] = t_cum
    
    print('--')
    print(t_cum)
    print('--')
    print(t_report[report_counter])
    print('--')
    print(dt)
    print('--')
    
    for inner_counter in range(0,4):

        for ix in range (0,Nx):
            for iy in range(0,Ny):
                
                if inner_counter == 0:
                    viscg[iz,iy,ix] = cal_viscg(p[iz,iy,ix],T_res,SGg)
                    rhog[iz,iy,ix] = cal_rhog(p[iz,iy,ix],T_res,SGg)
                    cg[iz,iy,ix] = cal_cg(p[iz,iy,ix],T_res,SGg)
                    Bg[iz,iy,ix] = cal_Bg(p[iz,iy,ix],T_res,SGg)
                    phi[iz,iy,ix] = cal_phi(p[iz,iy,ix],p_ref,phi_ref[iz,iy,ix],cr)
                    Jw[0,3,3] = Jw_cal(dx[0,3,3],dy[0,3,3],kx[0,3,3],ky[0,3,3],h[0,3,3],viscg[0,3,3],Bg[0,3,3],rw_3_3,0)
                    p_inner = p
                else:
                    viscg[iz,iy,ix] = cal_viscg(p_inner[iz,iy,ix],T_res,SGg)
                    rhog[iz,iy,ix] = cal_rhog(p_inner[iz,iy,ix],T_res,SGg)
                    cg[iz,iy,ix] = cal_cg(p_inner[iz,iy,ix],T_res,SGg)
                    Bg[iz,iy,ix] = cal_Bg(p_inner[iz,iy,ix],T_res,SGg)
                    phi[iz,iy,ix] = cal_phi(p_inner[iz,iy,ix],p_ref,phi_ref[iz,iy,ix],cr)
                    Jw[0,3,3] = Jw_cal(dx[0,3,3],dy[0,3,3],kx[0,3,3],ky[0,3,3],h[0,3,3],viscg[0,3,3],Bg[0,3,3],rw_3_3,0)
        print(p_inner)

        
        #Jacobian generator
        for ix in range (0,Nx):
            for iy in range(0,Ny):

                im = id_tran(iy,ix)
                if null[iz,iy,ix] == 0:
                    null_im.append(im)
                A_matrix_R[im,im] = dR_dp(iy,ix,'C')

                if (im-1)>= 0:
                    A_matrix_R[im,im-1] = dR_dp(iy,ix,'W')

                if (im+1)<= (Nx*Ny-1):
                    A_matrix_R[im,im+1] = dR_dp(iy,ix,'E')

                if (im-Nx)>= 0:
                    A_matrix_R[im,im-Nx] = dR_dp(iy,ix,'N')

                if (im+Nx)<= (Nx*Ny-1):
                    A_matrix_R[im,im+Nx] = dR_dp(iy,ix,'S')

                B_matrix_R[im,0] = R(iy,ix,'0')
#                 if abs(B_matrix_R[im,0]) > 0.01:
#                     print(B_matrix_R[im,0])
#                     print(iy)
#                     print(ix)
#         print(B_matrix_R)    

        null_im.sort()
        #print(null_im)
        A_matrix_1_R = np.delete(A_matrix_R,null_im,0)
        A_matrix_2_R = np.delete(A_matrix_1_R,null_im,1)
        B_matrix_1_R = np.delete(B_matrix_R,null_im,0)  
        
        soln_matrix_R = np.dot(np.linalg.inv(A_matrix_2_R),B_matrix_1_R)

        rev_counter = 0
        for iy in range (0,Ny):
            for ix in range(0,Nx):

                if null[iz,iy,ix] == 0:
                    residual_dp[iz,iy,ix] = 0
                else:
                    residual_dp[iz,iy,ix] = soln_matrix_R[rev_counter,0]
                    rev_counter += 1
   
        p_inner =  p_inner - residual_dp
        
        # MBE check
        IMB_num = 0
        IMB_deno = 0
        for ix in range (0,Nx):
            for iy in range(0,Ny):
                if (null[iz,iy,ix] != 0):
                    IMB_num += dx[iz,iy,ix]*dy[iz,iy,ix]*h[iz,iy,ix]/calpha*(cal_phi(p_inner[iz,iy,ix],p_ref,phi_ref[iz,iy,ix],cr)/cal_Bg(p_inner[iz,iy,ix],T_res,SGg)-cal_phi(p[iz,iy,ix],p_ref,phi_ref[iz,iy,ix],cr)/cal_Bg(p[iz,iy,ix],T_res,SGg))
                if (qgsc[iz,iy,ix] != 0):
                    IMB_deno += dt*qgsc[iz,iy,ix]

                if (Jw[iz,iy,ix] != 0 ):
                    IMB_deno += -dt*Jw[iz,iy,ix]*(p_inner[iz,iy,ix]-pwf[iz,iy,ix])
                

        IMB = IMB_num/IMB_deno

        #print(p_new)
        print(IMB)
        #print(p_inner)
        print('')    
    
    p_new = p_inner #assign the last inner iteration to p_new
    p_28[report_counter] = p_new[0,2,8]
    Jw_28[report_counter] = Jw_cal(dx[0,2,8],dy[0,2,8],kx[0,2,8],ky[0,2,8],h[0,2,8],viscg[0,2,8],Bg[0,2,8],rw_2_8,0)
    pwf_28[report_counter] = qgsc[0,2,8]/Jw_28[report_counter] + p_28[report_counter]
    
    p_33[report_counter] = p_new[0,3,3]
    q_33[report_counter] = Jw[0,3,3]*(p_new[0,3,3]-pwf[0,3,3])
    Jw_33[report_counter] = Jw[0,3,3]
    p_res_average[report_counter] = p_res_average_cal(p_new)
    MB_error[report_counter] = IMB
    
    report_counter += 1
    
    p[:] = p_new[:]

--
0.125
--
0.125
--
0.125
--
[[[   0.         7747.88679396 7747.88679396 7747.88679396    0.
      0.            0.            0.            0.            0.
      0.            0.        ]
  [7747.88679396 7747.88679396 7747.88679396 7747.88679396 7747.88679396
      0.            0.            0.         7747.88679396 7747.88679396
      0.            0.        ]
  [7747.88679396 7747.88679396 7747.88679396 7747.88679396 7747.88679396
   7747.88679396 7747.88679396 7747.88679396 7747.88679396 7747.88679396
   7747.88679396 7747.88679396]
  [7747.88679396 7747.88679396 7747.88679396 7747.88679396 7747.88679396
   7747.88679396 7747.88679396 7747.88679396 7747.88679396 7747.88679396
   7747.88679396 7747.88679396]
  [7747.88679396 7747.88679396 7747.88679396 7747.88679396 7747.88679396
   7747.88679396 7747.88679396 7747.88679396 7747.88679396 7747.88679396
      0.            0.        ]
  [   0.            0.            0.         7747.88679396 7747.88679396
   7747.88679396 7747.8

C:\Users\Kittipong Limchuchua\Anaconda3\lib\site-packages\ipykernel_launcher.py:90: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\Kittipong Limchuchua\Anaconda3\lib\site-packages\ipykernel_launcher.py:93: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\Kittipong Limchuchua\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in double_scalars
  if __name__ == '__main__':
C:\Users\Kittipong Limchuchua\Anaconda3\lib\site-packages\ipykernel_launcher.py:193: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\Kittipong Limchuchua\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\Kittipong Limchuchua\Anaconda3\lib\site-packages\ipykernel_launcher.py:182: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\Kittipong Limchuchua\Anaconda3\lib\site-packages\ipykernel_launcher.py:122: RuntimeWarning:

1.0036236217610954

[[[   0.         7747.88679322 7747.88678142 7747.88671567    0.
      0.            0.            0.            0.            0.
      0.            0.        ]
  [7747.88679265 7747.88676915 7747.88623067 7747.88062923 7747.88641444
      0.            0.            0.         7747.42908039 7747.88170326
      0.            0.        ]
  [7747.88676828 7747.88609853 7747.86517239 7747.43295505 7747.87027183
   7747.88640214 7747.88571284 7747.70761508 7728.42583734 7747.71815625
   7747.88453706 7747.88677018]
  [7747.88650571 7747.87533664 7747.33304312 7719.68560848 7747.41522952
   7747.87947668 7747.88670648 7747.88183729 7747.47872753 7747.88215634
   7747.88669705 7747.88679277]
  [7747.88676892 7747.88613129 7747.86559338 7747.44496561 7747.87056199
   7747.8864263  7747.88679002 7747.88665646 7747.87596248 7747.88666751
      0.            0.        ]
  [   0.            0.            0.         7747.8797685  7747.88636733
   7747.88678173 7747.88679383 77

0.9969797431514965

[[[   0.         7747.88679323 7747.88678149 7747.88671617    0.
      0.            0.            0.            0.            0.
      0.            0.        ]
  [7747.88679266 7747.88676931 7747.88623424 7747.88066826 7747.88641684
      0.            0.            0.         7747.43108475 7747.88172556
      0.            0.        ]
  [7747.88676844 7747.88610295 7747.8653098  7747.43583212 7747.87037656
   7747.88640461 7747.88571753 7747.70838865 7728.51394653 7747.71888059
   7747.88454675 7747.88677028]
  [7747.88650754 7747.87540937 7747.33656321 7719.87200741 7747.41821923
   7747.87952306 7747.88670697 7747.88185889 7747.48050416 7747.8821765
   7747.88669747 7747.88679278]
  [7747.88676908 7747.8861355  7747.86572788 7747.44776289 7747.87066488
   7747.88642863 7747.88679004 7747.88665706 7747.87600964 7747.88666806
      0.            0.        ]
  [   0.            0.            0.         7747.87981295 7747.88637003
   7747.88678181 7747.88679383 774

1.0031431901893673

[[[   0.         7747.88678912 7747.88672334 7747.88642066    0.
      0.            0.            0.            0.            0.
      0.            0.        ]
  [7747.88678539 7747.88665516 7747.88415685 7747.86338224 7747.88499282
      0.            0.            0.         7746.57397699 7747.86719148
      0.            0.        ]
  [7747.88665023 7747.88354359 7747.80552477 7746.59053845 7747.82433229
   7747.88493752 7747.88261418 7747.37000142 7710.69289682 7747.40149033
   7747.8781539  7747.88667971]
  [7747.88544129 7747.84367981 7746.31428048 7693.77496213 7746.54190245
   7747.85902133 7747.88637609 7747.86790046 7746.73085149 7747.86912579
   7747.88633168 7747.88678711]
  [7747.88665265 7747.88367363 7747.80663786 7746.62371835 7747.82538264
   7747.88505613 7747.88677146 7747.88614294 7747.84616296 7747.88619442
      0.            0.        ]
  [   0.            0.            0.         7747.85993141 7747.88477293
   7747.8867246  7747.88679309 77

...

0.9989353896511165

[[[   0.         7747.43585791 7746.72469782 7746.29366401    0.
      0.            0.            0.            0.            0.
      0.            0.        ]
  [7747.1805949  7746.02514726 7741.72024635 7736.40355937 7741.84798128
      0.            0.            0.         7665.08026471 7735.09102729
      0.            0.        ]
  [7745.88445995 7740.59019793 7718.91309606 7670.27189511 7722.42312675
   7741.99871571 7744.46210218 7709.11423314 7498.43188002 7713.1902286
   7742.94383522 7747.33082771]
  [7744.05482365 7730.95844055 7663.7221628  7364.17826091 7670.18813791
   7734.79969575 7746.13484851 7738.32663195 7692.14439529 7739.0752608
   7746.14535037 7747.68114239]
  [7745.46487769 7739.43575152 7715.10179867 7669.87714319 7722.33580526
   7742.71404607 7747.3478212  7745.85933148 7734.77018645 7745.93699411
      0.            0.        ]
  [   0.            0.            0.         7732.48593944 7741.46015119
   7746.55449158 7747.76614095 7747

1.0012861219799616

[[[   0.         7747.19952451 7746.23720436 7745.70071071    0.
      0.            0.            0.            0.            0.
      0.            0.        ]
  [7746.82213818 7745.29185135 7739.89664275 7733.68160105 7739.8404988
      0.            0.            0.         7654.05217325 7731.7187082
      0.            0.        ]
  [7745.08152154 7738.46026032 7713.16822848 7660.25498127 7717.04713505
   7740.08136635 7743.43205504 7703.15263312 7486.48450358 7707.97053298
   7741.69367907 7747.11631237]
  [7742.78455943 7727.29254968 7653.48758732 7346.15361556 7660.37261284
   7731.77487321 7745.49368116 7736.05099277 7685.91510009 7736.99022662
   7745.53781477 7747.58010536]
  [7744.45132117 7736.81893231 7708.19292493 7659.44739455 7716.89023532
   7741.06273574 7747.0989313  7745.18940675 7732.28521695 7745.28292152
      0.            0.        ]
  [   0.            0.            0.         7728.61776308 7739.36089424
   7745.98365873 7747.69705232 7747

0.9990606445535255

[[[   0.         7747.19956889 7746.23736189 7745.70092217    0.
      0.            0.            0.            0.            0.
      0.            0.        ]
  [7746.82222166 7745.29215916 7739.89806295 7733.6843183  7739.84205314
      0.            0.            0.         7654.08011871 7731.72246413
      0.            0.        ]
  [7745.08185736 7738.4620499  7713.1776721  7660.27872059 7717.05555292
   7740.08280273 7743.43254217 7703.16167913 7486.51234887 7707.97757115
   7741.69434029 7747.11634916]
  [7742.78529014 7727.29698294 7653.51208222 7346.21347886 7660.3954221
   7731.77805542 7745.49390499 7736.05268335 7685.92447687 7736.99165891
   7745.5379964  7747.58011577]
  [7744.45182058 7736.82149614 7708.20599881 7659.47315503 7716.89892882
   7741.06391982 7747.09898566 7745.18962056 7732.28719986 7745.28312215
      0.            0.        ]
  [   0.            0.            0.         7728.62263894 7739.36264925
   7745.98387694 7747.6970606  774

1.0011446715850436

[[[   0.         7746.89379018 7745.6484222  7745.00152376    0.
      0.            0.            0.            0.            0.
      0.            0.        ]
  [7746.36447073 7744.42568489 7737.89920058 7730.8066036  7737.56965378
      0.            0.            0.         7643.52070243 7728.06994703
      0.            0.        ]
  [7744.12654284 7736.13527989 7707.38906547 7650.74385742 7711.55703407
   7737.92594298 7742.26925222 7697.27021635 7475.84560303 7702.825617
   7740.3495369  7746.86016632]
  [7741.33536412 7723.4950127  7643.82477387 7330.55052653 7651.07048675
   7728.60180576 7744.74241153 7733.64669409 7680.09266859 7734.78981815
   7744.83809705 7747.45093617]
  [7743.23243385 7733.92004322 7701.15471025 7649.45132495 7711.3183053
   7739.21624368 7746.78820863 7744.42903121 7729.79834545 7744.53723341
      0.            0.        ]
  [   0.            0.            0.         7724.48249173 7736.99850168
   7745.29213237 7747.60445753 7747.

0.999155096608363

[[[   0.         7746.8938491  7745.6486121  7745.00177233    0.
      0.            0.            0.            0.            0.
      0.            0.        ]
  [7746.3645829  7744.42605248 7737.90074019 7730.80939874 7737.57146245
      0.            0.            0.         7643.54601327 7728.07407506
      0.            0.        ]
  [7744.12695396 7736.13722675 7707.39824017 7650.76493129 7711.56542877
   7737.92759581 7742.26980685 7697.27888146 7475.86810939 7702.83233709
   7740.35023747 7746.86020904]
  [7741.33620826 7723.49947976 7643.84632326 7330.59612193 7651.09066188
   7728.60506392 7744.74267549 7733.6484755  7680.10079397 7734.79132032
   7744.83830972 7747.45094953]
  [7743.23306617 7733.92293446 7701.1677972  7649.47464784 7711.32700559
   7739.2175763  7746.78827576 7744.42927405 7729.80023224 7744.53746247
      0.            0.        ]
  [   0.            0.            0.         7724.48772805 7737.00052461
   7745.29240283 7747.60446854 774

In [6]:
from tabulate import tabulate
print_production_matrix = np.zeros((len(t_report),7))

for j in range(0,len(t_report)):
    print_production_matrix[j,0] = t_report[j]
    print_production_matrix[j,1] = p_28[j]
    print_production_matrix[j,2] = pwf_28[j]
    print_production_matrix[j,3] = Jw_28[j]
    print_production_matrix[j,4] = p_33[j]
    print_production_matrix[j,5] = q_33[j]
    print_production_matrix[j,6] = Jw_33[j]
    

print(tabulate(print_production_matrix, headers=['Time', 'P(2,8)','Pwf(2,8)','Jw(2,8)','P(3,3)','q(3,3)','Jw(3,3)']))

  Time    P(2,8)    Pwf(2,8)    Jw(2,8)    P(3,3)       q(3,3)    Jw(3,3)
------  --------  ----------  ---------  --------  -----------  ---------
 0       7747.89     7747.89     inf      7747.89  0               inf
 0.125   7728.51     6978.87    4001.92   7719.87  7.22949e+06    4203.5
 0.25    7710.77     6961.49    4003.87   7693.93  7.12555e+06    4206.51
 0.375   7694.47     6945.53    4005.67   7669.87  7.029e+06      4209.31
 0.5     7679.48     6930.85    4007.33   7647.51  6.93918e+06    4211.92
 0.75    7653.75     6905.66    4010.19   7608.49  6.78221e+06    4216.5
 1       7631.52     6883.89    4012.67   7574.27  6.64424e+06    4220.53
 1.25    7612.16     6864.94    4014.83   7544.09  6.52239e+06    4224.1
 1.5     7595.2      6848.32    4016.73   7517.35  6.41425e+06    4227.28
 2       7568.12     6821.81    4019.78   7474.21  6.23948e+06    4232.42
 2.5     7546.03     6800.18    4022.27   7438.91  6.09618e+06    4236.65
 3       7527.62     6782.16    4024.35   74

In [7]:
from tabulate import tabulate
print_report_matrix = np.zeros((len(t_report),3))

for j in range(0,len(t_report)):
    print_report_matrix[j,0] = t_report[j]
    print_report_matrix[j,1] = p_res_average[j]
    print_report_matrix[j,2] = MB_error[j]
    

print(tabulate(print_report_matrix, headers=['Time', 'Average P','MB error']))

  Time    Average P    MB error
------  -----------  ----------
 0          7756.34    0
 0.125      7747.1     0.99698
 0.25       7746.33    0.997407
 0.375      7745.57    0.99776
 0.5        7744.82    0.998051
 0.75       7743.35    0.996963
 1          7741.9     0.997586
 1.25       7740.48    0.998042
 1.5        7739.07    0.99838
 2          7736.3     0.997626
 2.5        7733.58    0.998168
 3          7730.89    0.998523
 3.5        7728.23    0.998764
 4          7725.59    0.998935
 4.5        7722.98    0.999061
 5          7720.38    0.999155


In [8]:
print(dt_report)
print(p)

[0, 0.125, 0.125, 0.125, 0.125, 0.25, 0.25, 0.25, 0.25, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
[[[   0.         7746.8938491  7745.6486121  7745.00177233    0.
      0.            0.            0.            0.            0.
      0.            0.        ]
  [7746.3645829  7744.42605248 7737.90074019 7730.80939874 7737.57146245
      0.            0.            0.         7643.54601327 7728.07407506
      0.            0.        ]
  [7744.12695396 7736.13722675 7707.39824017 7650.76493129 7711.56542877
   7737.92759581 7742.26980685 7697.27888146 7475.86810939 7702.83233709
   7740.35023747 7746.86020904]
  [7741.33620826 7723.49947976 7643.84632326 7330.59612193 7651.09066188
   7728.60506392 7744.74267549 7733.6484755  7680.10079397 7734.79132032
   7744.83830972 7747.45094953]
  [7743.23306617 7733.92293446 7701.1677972  7649.47464784 7711.32700559
   7739.2175763  7746.78827576 7744.42927405 7729.80023224 7744.53746247
      0.            0.        ]
  [   0.            0.            0

In [9]:
import numpy as np
import matplotlib.pyplot as plt

cor = []

x_cor = 0
for ix in range (0,Nx):
    if ix == 0:
        x_cor += dx[0,ix]/2    
    else:
        x_cor += dx[0,ix-1]/2 + dx[0,ix]/2
    y_cor = 0
    for iy in range (0,Ny):
        if iy == 0:
            y_cor += dy[iy,0]/2  
            cor.append([x_cor,y_cor])
        else:
            y_cor += dy[iy,0]/2 + dy[iy-1,0]/2
            cor.append([x_cor,y_cor])
                
    
cor_array = np.array(cor)

pressure = []

for ix in range (0,Nx):
    for iy in range (0,Ny):
        pressure.append(p_new[iy,ix])

pressure_array = np.array(pressure)

grid_x, grid_y = np.mgrid[0:x_cor:100j, 0:y_cor:100j]

from scipy.interpolate import griddata
grid_z0 = griddata(cor_array, pressure_array, (grid_x, grid_y), method='nearest')

plt.imshow(grid_z0.T, extent=(0,x_cor,y_cor,0),vmin=6500, vmax=7750,cmap="PuBu")
plt.title('well block pressure')
plt.colorbar()
plt.show()

IndexError: index 3 is out of bounds for axis 0 with size 3

In [ ]:
A = [[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16]]
A_del = np.delete(A,[1,3],0)
A_del = np.delete(A_del,[1,3],1)
print(A_del)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

cor = []

x_cor = 0
for ix in range (0,Nx):
    if ix == 0:
        x_cor += dx[0,ix]/2    
    else:
        x_cor += dx[0,ix-1]/2 + dx[0,ix]/2
    y_cor = 0
    for iy in range (0,Ny):
        if iy == 0:
            y_cor += dy[iy,0]/2  
            cor.append([x_cor,y_cor])
        else:
            y_cor += dy[iy,0]/2 + dy[iy-1,0]/2
            cor.append([x_cor,y_cor])
                
    
cor_array = np.array(cor)

porosity = []

for ix in range (0,Nx):
    for iy in range (0,Ny):
        porosity.append(phi[iy,ix])

porosity_array = np.array(porosity)

grid_x, grid_y = np.mgrid[0:x_cor:100j, 0:y_cor:100j]

from scipy.interpolate import griddata
grid_z0 = griddata(cor_array, porosity_array, (grid_x, grid_y), method='nearest')

plt.imshow(grid_z0.T, extent=(0,x_cor,y_cor,0),vmin=0.18, vmax=0.24,cmap="PuBu_r")
plt.title('porosity')
plt.colorbar()
plt.show()

In [ ]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

x_array = []
y_array = []
z_array = []

x_cor = 0
for ix in range (0,Nx):
    if ix == 0:
        x_cor += dx[0,ix]/2 
        x_array.append(x_cor)
    else:
        x_cor += dx[0,ix-1]/2 + dx[0,ix]/2
        x_array.append(x_cor)

y_cor = 0
for iy in range (0,Ny):
    if iy == 0:
        y_cor += dy[iy,0]/2  
        y_array.append(y_cor)
    else:
        y_cor += dy[iy,0]/2 + dy[iy-1,0]/2
        y_array.append(y_cor)


for ix in range (0,Nx):
    for iy in range (0,Ny):
        z_array.append(middle[iy,ix])


X, Y = np.meshgrid(x_array, y_array)
Z = -middle
pressure = p_new
Z[abs(Z)<1] = np.nan
    
fig = plt.figure()

ax = Axes3D(fig)
#ax = fig.add_subplot(111, projection='3d')
ax.view_init(45,45)
ax.set_xlim3d(abs(np.max(X)),0)
ax.set_ylim3d(abs(np.max(Y)),0)
ax.set_zlim3d(-9450,-9250)
# here we create the surface plot, but pass V through a colormap
# to create a different color for each patch
norm = matplotlib.colors.Normalize(vmin=6500, vmax=7750)
ax.plot_surface(X, Y, Z, facecolors = cm.viridis(norm(pressure)),linewidth=0.1)
#fig.colorbar(fig,shrink=0.5, aspect=5)
plt.title('Pressure Distribution 3D',fontsize='14', y=1.1)
plt.xlabel('x (ft)')
plt.ylabel('y (ft)')

pressure_range = np.arange(6500,7750)
m = cm.ScalarMappable(cmap=cm.viridis)
m.set_array(pressure_range)
plt.colorbar(m)

In [ ]:
import numpy as np

z = np.arange(20).reshape(5, 4)
mask = np.zeros(z.shape, dtype=bool)

mask[3, 2] = True

print(z)
print(np.ma.masked_array(z, mask))

In [ ]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

x_array = []
y_array = []
z_array = []

x_cor = 0
for ix in range (0,Nx):
    if ix == 0:
        x_cor += dx[0,ix]/2 
        x_array.append(x_cor)
    else:
        x_cor += dx[0,ix-1]/2 + dx[0,ix]/2
        x_array.append(x_cor)

y_cor = 0
for iy in range (0,Ny):
    if iy == 0:
        y_cor += dy[iy,0]/2  
        y_array.append(y_cor)
    else:
        y_cor += dy[iy,0]/2 + dy[iy-1,0]/2
        y_array.append(y_cor)


for ix in range (0,Nx):
    for iy in range (0,Ny):
        z_array.append(middle[iy,ix])


X, Y = np.meshgrid(x_array, y_array)
Z = -middle
porosity = phi
Z[abs(Z)<1] = np.nan
    
fig = plt.figure()

ax = Axes3D(fig)
#ax = fig.add_subplot(111, projection='3d')
ax.view_init(45,45)
ax.set_xlim3d(abs(np.max(X)),0)
ax.set_ylim3d(abs(np.max(Y)),0)
ax.set_zlim3d(-9450,-9250)
# here we create the surface plot, but pass V through a colormap
# to create a different color for each patch
norm = matplotlib.colors.Normalize(vmin=0.18, vmax=0.25)
ax.plot_surface(X, Y, Z, facecolors = cm.PuBu_r(norm(porosity)),linewidth=0.1)
#fig.colorbar(fig,shrink=0.5, aspect=5)
plt.title('Porosity Distribution 3D',fontsize='14', y=1.1)
plt.xlabel('x (ft)')
plt.ylabel('y (ft)')

porosity_range = np.arange(0.18,0.25,0.01)
m = cm.ScalarMappable(cmap=cm.PuBu_r)
m.set_array(porosity_range)
plt.colorbar(m)

In [ ]:
R_matrix = np.zeros((Ny,Nx))
for iy in range (0,Ny):
    for ix in range (0,Nx):
        R_matrix[iy,ix] = R(iy,ix,'0')
        
R_matrixW = np.zeros((Ny,Nx))
for iy in range (0,Ny):
    for ix in range (0,Nx):
        R_matrixW[iy,ix] = R(iy,ix,'W')
        
print(R_matrix)
print(R_matrixW)
print(null)

In [ ]:
dRdp_matrixC = np.zeros((Ny,Nx))
for iy in range (0,Ny):
    for ix in range (0,Nx):
        dRdp_matrixC[iy,ix] = dR_dp(iy,ix,'C')
print(dRdp_matrixC)

dRdp_matrixW = np.zeros((Ny,Nx))
for iy in range (0,Ny):
    for ix in range (0,Nx):
        dRdp_matrixW[iy,ix] = dR_dp(iy,ix,'W')
print(dRdp_matrixW)

In [ ]:
print(residual_dp)